<a href="https://colab.research.google.com/github/oea75/Image_classification/blob/master/Credit%20score%20model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Credit score modelling using machine learning

In [ ]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import files
uploaded = files.upload()

Saving loan.csv to loan.csv


KeyboardInterrupt: ignored

In [ ]:
loans = pd.read_csv(df, infer_datetime_format = True)

In [ ]:
loans.dtypes

In [ ]:
loans.shape

In [ ]:
loans.head(5)

In [ ]:
datecols = ['issue_d','last_pymnt_d','next_pymnt','last_credit__pull_d','final_d','earliest_cr_line']

In [ ]:
loans[datecols].dtypes

In [ ]:
import datetime
def getDate(s):
  s=str(s)[0:8]
  retval = None
  try:
    retval = datetime.datetime.strptime(s, "%Y%m%d")
  except:
    pass
    #print s

  return retval

In [ ]:
%%time
newcols = ['issue_date','final_date']
loans['issue_date']=loans['final_date'].apply(getDate)
loans['final_date']=loans['final_d'].apply(getDate)

In [ ]:
newcols = ['issue_date','final_date']
loans[datecols+newcols].head(10)

In [ ]:
loans[datecols+newcols].dtypes

In [ ]:
%%time
loans['outcome']=loans['final_date'] < datetime.datetime(2015,3,31)
loans['issue_year']=loans['issue_date'].apply(lambda x:x.year)
loans['rem_prncp'] = loans['funded_amnt'] - loans['total_rec_prncp']
loans['net_return'] = loans['total_pymnt']-loans['funded_amnt']

In [ ]:
loans[['funded_amnt','total_rec_prncp','rem_prncp','total_pymnt','funded_amt','net_return']].head(10)

Portfolio status

In [ ]:
loans.groupby(['outcome'])['outcome'].count()

In [ ]:
loans.groupby[(outcome, 'issue_year')]['outcome'].count()

In [ ]:
loans[(loans['outcome']==True)]['issue_date'].max()
max_issue_date

In [ ]:
loans.groupby(['inactive_loans'])['inactive_loans'].count()

In [ ]:
loans.groupby(['outcome', 'inactive_loans'])['id'].count()

**Building a credit scoring model**

In [ ]:
features = ['grade',                    # loan grade (categorical)
            'sub_grade_num',            # sub-grade of the loan as a number from 0 to 1
            'short_emp',                # one year or less of employment
            'emp_length_num',           # number of years of employment
            'home_ownership',           # home ownership status: own, mortgage or rent
            'dti',                      # debt to income ratio
            'purpose',                  # the loan purpose
            'payment_inc_ratio',        # ratio of the monhly payment to income
            'delinq_2yrs',              # number of delinquincies
            'delinq_2yrs_zero',         # no delinquincies in last 2 years
            'inq_last_6mths',           # number of creditor inquiries in last 6 months
            'last_delinq_none',         # has borrower had a delinquincy
            'last_major_derog_none',    # has borrower had 90 day or worse rating
            'open_acc',                 # number of open credit accounts
            'pub_rec',                  # number of derogatory public records
            'pub_rec_zero,',            # no derogatory public records
            'revol_util',               # percent of available credit being
            'total_rec_late_fee',       # total late fees received to day
            ]
response = 'bad_loans'

In [ ]:
# Get a proper subset of loans
completed_loans=loans[(loans['outcome']==1) & (loans['inactive_loans']==1)]

In [ ]:
clean_data=completed_loans[features+[response]].dropna()

In [ ]:
completed_loans.shape, clean_data.shape

In [ ]:
#load libraries
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelBinarizer
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve, auc, confusion_matrix

import sklearn
from sklearn_pandas import DataFrameMapper

numerical_cols = ['sub_grade_num', 'short_emp','emp_length','dti','payment_inc_ratio',
                  'delinq_2yrs','delinq_2yrs_zero','inq_last_6mths','last_delinq_none',
                  'last_major_derog_none','open_acc','pub_rec','pub_rec_zero',
                  'revol_util','total_rec_late_fee']

categorical_cols = ['grade','home_ownership','purpose']

mapper = DataFraleMapper([
                          ('grade',sklearn.preprocessing.LabelBinarizer()),
                          ('home_ownership', sklearn.preprocessing.LabelBinarizer()),
                          ('purpose', sklearn.preprocessing.LabelBinarizer()), ])

X1 = mapper.fit_transform(clean_data)
X1.shape

X2 = np.array(clean_data[numerical_cols])
X2.shape

In [ ]:
X = np.hstack((X1,X2)) # combine X1 & X2 side by side, i.e stacks them horizontally
y= np.array(clean_data['bad_loans'])

X.shape, y.shape

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=100)

In [ ]:
rf_lm = LogisticRegression()
rf_lm.fit(X_train, y_train)

In [ ]:
y_pred = rf_lm.predict_proba(X_test)[:,1]
y_pred_class=rf_lm.predict(X_test)
accuracy = np.sum(y_pred==y_test)/y_test.shape[0]

print('accuracy = %0.2f' %accuracy)

In [ ]:
good=np.sum([y==0])
bad=np.sum([y==1])
pct_bad=(bad/(bad+good))
print("Bad loans percentage = %0.2f" %pct_bad)

In [ ]:
cnf_matric = confusion_matrix(y_test, y_pred_class)
print(cnf_matrix)

In [ ]:
# our model assum a 50% probability cutoff. Let's take another percentage

In [ ]:
y_pred_class2 = y_pred>.7
cnf_matrix = confusion_matrix(y_test, y_pred_class2)
print(cnf_matrix)

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
y_pred_prob=rf_lm.predict_proba(X_test)[:1]
fpr_rf_lm, tpr_rf_lm,_= roc_curve(y_test, y_pred_prob)

plt.figure(1)
plt.plot([0,1], [0,1], 'k--')
plt.xlabel('False positive rate')
plt.ylabel('True positive rate')
plt.title('ROC curve')
plt.legend(loc='best')
plt.show()

In [ ]:
#Deploying the model for use
new_case = clean_data.loc[1986:1987] # Here we can select any past record
new_case

In [ ]:
def prep(case):
  X1=np.array(case[numerical_cols])
  X2=mapper.transform(case)
  return(np.hstack((X1,X2)))

In [ ]:
num_features=np.array(new_case[numerical_cols])
cat_features=mapper.transform(new_case)
inp=np.hstack((num_features, cat_features))

In [ ]:
rf_lm.predict_proba(inp)

In [ ]:
rf_lm.predict(prep(new_case))